## Towards a final dataset

- Create final dataset from existing scripts

Environment

In [1]:
import pandas as pd
import dateutil
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import matplotlib as plt
import numpy as np

Original data

In [366]:
data = pd.read_csv('../20190313_TW_MigrantMigration.csv')
data[0:1]

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,user_location,status_url,entities_str
0,1.102720e+18,darleneclarke5,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:47 +0000 2019,04/03/2019 23:59:47,NaN,en,NaN,NaN,564257648.0,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://abs.twimg.com/sticky/default_profile_im...,197,579.0,NaN,http://twitter.com/darleneclarke5/statuses/110...,"{""hashtags"":[],""symbols"":[],""user_mentions"":[{..."


In [367]:
#New df
df=data

Current state of the dataframe

In [409]:
df[0:100]

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLang_new,authorLocation,authorLocation_new,authorPicture,authorPicture_new,...,tweetDate_new,tweetID,tweetID_original,tweetMethod,tweetMethod_new,tweetSentiment,tweetSentiment_cat,tweetText,tweetTimeDate,tweetURL
0,197,579.0,5.642576e+08,NaN,en,English,NaN,NaN,http://abs.twimg.com/sticky/default_profile_im...,Default,...,2019-03-04,0,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Web_Standard,-0.9403,Very Negative,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:47 +0000 2019,http://twitter.com/darleneclarke5/statuses/110...
1,940,225.0,3.109106e+09,NaN,en,English,top your momma,Not USA,http://pbs.twimg.com/profile_images/1062099415...,Personalized,...,2019-03-04,1,NaN,"<a href=""http://twitter.com/download/android"" ...",Smartphone,-0.3612,Negative,RT @1776Stonewall: https://t.co/CyucESMlFM Te...,Mon Mar 04 23:59:40 +0000 2019,http://twitter.com/BlueelvisEric/statuses/1102...
2,103,275.0,1.083790e+18,NaN,en,English,NaN,NaN,http://pbs.twimg.com/profile_images/1083792878...,Personalized,...,2019-03-04,2,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Web_Lite,0.6908,Positive,RT @EmeraldRobinson: Trump's base was promised...,Mon Mar 04 23:59:38 +0000 2019,http://twitter.com/Potus16Abe/statuses/1102720...
3,718,869.0,7.967720e+17,NaN,en,English,NaN,NaN,http://pbs.twimg.com/profile_images/1061628259...,Personalized,...,2019-03-04,3,NaN,"<a href=""http://twitter.com/download/android"" ...",Smartphone,-0.1531,Negative,RT @AnjaKolibri: Australian humpback #whale po...,Mon Mar 04 23:59:34 +0000 2019,http://twitter.com/soozbo/statuses/11027202687...
4,666,481.0,1.056307e+08,NaN,en,English,Netherlands,Not USA,http://pbs.twimg.com/profile_images/3788000007...,Personalized,...,2019-03-04,4,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Smartphone,-0.8316,Very Negative,RT @NinaDSchick: In what should come as no sur...,Mon Mar 04 23:59:32 +0000 2019,http://twitter.com/jp_dutch/statuses/110272026...
5,125,232.0,1.078316e+09,NaN,en,English,NaN,NaN,http://pbs.twimg.com/profile_images/6961402810...,Personalized,...,2019-03-04,5,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Web_Standard,-0.9403,Very Negative,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:28 +0000 2019,http://twitter.com/RJ_Kendall/statuses/1102720...
6,260,298.0,2.491415e+07,NaN,en,English,NaN,NaN,http://pbs.twimg.com/profile_images/8623125151...,Personalized,...,2019-03-04,6,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Smartphone,0.1573,Positive,RT @clairlemon: A Swedish police-officer--who ...,Mon Mar 04 23:59:26 +0000 2019,http://twitter.com/gabrieleverett/statuses/110...
7,334,521.0,7.150480e+17,NaN,ja,Other,F研 なのるな Attellui,Not USA,http://pbs.twimg.com/profile_images/1096403448...,Personalized,...,2019-03-04,7,NaN,"<a href=""http://twitter.com/download/android"" ...",Smartphone,0.0000,Neutral,"RT @minajirou: Sir.Fennek/Kite\nCuckoo/York,Ne...",Mon Mar 04 23:59:19 +0000 2019,http://twitter.com/lT1g0n8T2/statuses/11027202...
8,240,623.0,8.066630e+17,NaN,fi,Other,NaN,NaN,http://pbs.twimg.com/profile_images/9758078583...,Personalized,...,2019-03-04,8,NaN,"<a href=""http://twitter.com/download/android"" ...",Smartphone,0.7579,Positive,"RT @Charles_Lister: ""The far-right #AfD, which...",Mon Mar 04 23:59:18 +0000 2019,http://twitter.com/suse_______/statuses/110272...
9,2754,2551.0,9.819028e+07,NaN,de,Other,Bleckede,Not USA,http://pbs.twimg.com/profile_images/9815630205...,Personalized,...,2019-03-04,9,NaN,"<a href=""https://www.ffd365.de"" rel=""nofollow""...",Other,-0.5994,Negative,"#FFD365 Verschwiegene Migration, Heißer Empfan...",Mon Mar 04 23:59:11 +0000 2019,http://twitter.com/ffd365/statuses/11027201728...


Unique ID variable for each tweet

In [369]:
df.reset_index(inplace=True)
df.rename(columns={'index':'tweetID'}, inplace=True)

Sentiment Score

In [370]:
#create an object called map_ID_text
#containing a dictionary
#with a tuple for each row in the df where
#element 1 of the tuple is the value in tweet ID
#and element 2 of the tuple is the value in text

map_ID_text = dict((tuple(x) for x in df[['tweetID', 'text']].values))

In [371]:
#create an object called analyzer
#containing the SentimentIntensityAnalyzer() function
#create an object called map_ID_valence_dict
#containing an empty dictionary
#loop through items in map_ID_text
#calculate sentiment score (that is, valence) of each text
#append score to map_ID_valence_dict, indexed by ID

analyzer = SentimentIntensityAnalyzer()

map_ID_valence_dict = {}

for ID, text in map_ID_text.items():
    map_ID_valence_dict[ID] = analyzer.polarity_scores(text)

In [372]:
#create an object called map_ID_senti_score
#containing an empty dictionary
#loop through items in map_ID_valence_dict
#containing an empty dictionary
#for pair in map_ID_text
#append "compound" element of valence to map_ID_valence_dict, indexed by ID

map_ID_senti_score = {} # Mapping IDs to a single score 'combined'

for ID, val in map_ID_valence_dict.items():
    map_ID_senti_score[ID] = val['compound']

In [373]:
#create new column for sentiment score
#fill with items from map_ID_senti_score
df['SentiScore_VADER'] = df['tweetID'].map(map_ID_senti_score)

Drop unnecessary columns

In [375]:
#Drop the "text" and "entities" columns that Sunyam is working on. 
#Also dropping time because it is redundant with "created_at"
df = df.drop(['id_str','time','entities_str', 'geo_coordinates'], axis=1)

In [376]:
#Rename columns
df = df.rename(columns={"id_str":"tweetID",
                        "from_user":"authorScreen",
                        "created_at":"tweetTimeDate",
                        "geo_coordinates":"tweetGeo",
                        "user_lang":"authorLang",
                        "in_reply_to_user_id_str":"authorID_original",
                        "in_reply_to_screen_name":"authorScreen_original",
                        "from_user_id_str":"authorID",
                        "in_reply_to_status_id_str":"tweetID_original",
                        "source":"tweetMethod", 
                        "profile_image_url":"authorPicture",
                        "user_followers_count":"authorFollower",
                        "user_friends_count":"authorFriend",
                        "user_location":"authorLocation",
                        "status_url":"tweetURL",
                        "text":"tweetText",
                        "SentiScore_VADER": "tweetSentiment"})

In [408]:
#Group author and tweet variables together in the dataset.
df = df.sort_index(axis=1)

Recode variables

In [378]:
df["authorLang_new"] = df["authorLang"].astype('category')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'en.*', value='English')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'^((?!English)\S*)', value='Other')
df['authorLang_new'].value_counts()

English    109347
Other       14177
Name: authorLang_new, dtype: int64

In [379]:
df["authorPicture_new"] = df["authorPicture"].astype('category')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*default_profile_images\S*', value='Default')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*/profile_images\S*', value='Personalized')
df['authorPicture_new'].value_counts()

Personalized    117131
Default           6393
Name: authorPicture_new, dtype: int64

In [380]:
df["tweetMethod_new"] = df["tweetMethod"].astype('category')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPhone\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for Android\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web Client\S*', value='Web_Standard')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web App\S*', value='Web_Lite')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPad\S*', value='Web_Lite')
df['tweetMethod_new'].value_counts()[0:10]

Smartphone                                                                             74524
Web_Standard                                                                           24144
Web_Lite                                                                               16764
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                   1300
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>     1079
<a href="https://ifttt.com" rel="nofollow">IFTTT</a>                                     500
<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>                                 474
<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>                    457
<a href="https://buffer.com" rel="nofollow">Buffer</a>                                   360
<a href="http://publicize.wp.com/" rel="nofollow">WordPress.com</a>                      266
Name: tweetMethod_new, dtype: int64

In [381]:
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*href.*', value='Other')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^Web_Standard.*(T1).*', value='Other')
df['tweetMethod_new'].value_counts()

Smartphone      74524
Web_Standard    24144
Web_Lite        16764
Other            8092
Name: tweetMethod_new, dtype: int64

In [382]:
df['tweetDate'] = df['tweetTimeDate'].replace(regex=r'\s\d{2}:.*:\d{2}\s.{5}', value='')
df['tweetDate_new'] = df['tweetDate'].apply(dateutil.parser.parse, dayfirst=True)
df['tweetDate_new'].value_counts()[0:10]

2019-02-28    2913
2019-01-26    2910
2019-02-22    2909
2019-02-10    2906
2019-02-04    2905
2019-02-08    2902
2019-02-11    2901
2019-01-28    2900
2019-03-01    2897
2019-02-23    2896
Name: tweetDate_new, dtype: int64

In [384]:
#Recode the numerical sentiment scores into categorical data
bins = [-1, -0.8, -0.05, 0.05, 0.8, 1]
names = ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
df['tweetSentiment_cat'] = pd.cut(df['tweetSentiment'], bins, labels=names)
df['tweetSentiment_cat'].value_counts()

Negative         46370
Neutral          27895
Positive         27009
Very Negative    16475
Very Positive     5775
Name: tweetSentiment_cat, dtype: int64

Author locations

In [399]:
df['authorLocation_new'] = df['authorLocation']
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*USA.*', value='United States')
df['authorLocation_new'].value_counts()[0:9]

United States              15625
Washington, DC               680
London                       652
London, England              648
Canada                       561
United Kingdom               530
England, United Kingdom      516
Los Angeles, CA              476
New York, NY                 475
Name: authorLocation_new, dtype: int64

In [400]:
USA = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
USA[0:10]

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific
5,Colorado,CO,West,Mountain
6,Connecticut,CT,Northeast,New England
7,District of Columbia,DC,South,South Atlantic
8,Delaware,DE,South,South Atlantic
9,Florida,FL,South,South Atlantic


In [401]:
states = USA['State'].tolist()
print(states)

['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']


In [403]:
for x in states:
    df['authorLocation_new'] = df['authorLocation_new'].replace(regex='.*' + str(x) + '.*', value='United States')
df['authorLocation_new'][df['authorLocation_new']!="United States"].value_counts()[0:9]

London                     652
London, England            648
Canada                     561
United Kingdom             530
England, United Kingdom    516
Los Angeles, CA            476
Australia                  391
UK                         375
France                     339
Name: authorLocation_new, dtype: int64

In [397]:
stateCodes = USA['State Code'].tolist()
print(stateCodes)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [404]:
for x in stateCodes:
    df['authorLocation_new'] = df['authorLocation_new'].replace(regex='^.*' + str(x) + '.*$', value='United States')
df['authorLocation_new'][df['authorLocation_new']!="United States"].value_counts()[0:9]

London                     652
London, England            648
Canada                     561
United Kingdom             530
England, United Kingdom    516
Australia                  391
UK                         375
France                     339
Earth                      298
Name: authorLocation_new, dtype: int64

In [405]:
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Chicago.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NYC.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Los Angeles.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Francisco.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Boston.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^United States of America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NC$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^US$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NJ$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^CA$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^TEXAS$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Seattle$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Atlanta$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Diego$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Philadelphia$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Midwest$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Pacific Northwest$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Brooklyn$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^florida$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Las Vegas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^usa.*$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Bay Area$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^texas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Jose$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Miami$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^So Cal$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Tucson$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^The South$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Dallas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^nyc$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^usa$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^los angeles$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Houston$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Tx$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Upstate NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Usa$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Silicon Valley$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Troy, MICHIGAN$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^THE SOUTH$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Minneapolis', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Western PA', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^SF Bay Area', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Columbus', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^chicago$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Conway Ar$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Denver$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Nashville$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Puget Sound$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^new york$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Pittsburgh$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Commiefornia$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*\bUS\b.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Rocky Mountains.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Manhattan.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Vero Beach.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*california.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^ohio$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Western NY.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Southern CA.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*NEW YORK.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States of America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Trump Nation.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NY.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'United Sates of America', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'From Sea to Shining Sea', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'Portland OR', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States\..*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Merica.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*The Dirty South\, U\.S\.A\..*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Cali$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^In.*Ca\.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Buffalo NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Philly$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^wisconsin$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^New Orleans$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Long Island$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^los angeles, ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^.*Nashville.*', value='United States')

In [406]:
filter1 = df['authorLocation_new']=="United States"
filter2 = df['authorLocation_new'].isnull()
df['authorLocation_new'].where(filter1 | filter2, "Not USA", inplace = True) 
df['authorLocation_new'].value_counts()

Not USA          42788
United States    42418
Name: authorLocation_new, dtype: int64

In [410]:
df.to_csv('../20190403_AM_Data.csv',index=False)